In [1]:
import pandas as pd

# Load main database
enforce = pd.read_csv("../data/raw/enforcement.csv")

# Drop unnecessary columns
enforce = enforce.drop(['FACID', 'LTC', 'FAC_TYPE_CODE', 'PENALTY_TYPE', 'TOTAL_PENALTY_OFFSET_AMOUNT',
                        'TOTAL_COLLECTED_AMOUNT', 'TOTAL_BALANCE_DUE',
                        'ACLAIMS_VISIT_NUMBER', 'EVENTID', 'DEATH_RELATED', 'INTAKEID_ALL', 'SFY',
                        'APPEAL_DUE_DATE', 'APPEAL_RECEIVED_DATE', 'PRIORITY_ALL',
                        'PENALTY_CATEGORY_OTHER', 'VIOLATION_FROM_DATE', 'VIOLATION_TO_DATE'], axis=1)
enforce.head()

,FACILITY_NAME,FAC_FDR,DISTRICT_OFFICE,PENALTY_ISSUE_DATE,PENALTY_NUMBER,DISPOSITION,PENALTY_DETAIL,PENALTY_CATEGORY,APPEALED,CLASS_ASSESSED_INITIAL,CLASS_ASSESSED_FINAL,TOTAL_AMOUNT_INITIAL,TOTAL_AMOUNT_DUE_FINAL
0,STANFORD HEALTH CARE - VALLEYCARE,GENERAL ACUTE CARE HOSPITAL,East Bay,22Aug2007,20004196,Closed,AP - Immediate Jeopardy (HSC 1280.3),AE12 - Medication error,Yes,AP IJ,NaN,"$25,000","$25,000"
1,WASHINGTON HOSPITAL,GENERAL ACUTE CARE HOSPITAL,East Bay,06Nov2007,20004353,Closed,AP - Immediate Jeopardy (HSC 1280.3),AE12 - Medication error,No,AP IJ,NaN,"$25,000","$25,000"
2,EDEN MEDICAL CENTER,GENERAL ACUTE CARE HOSPITAL,East Bay,10Jan2008,20004575,Closed,Failure to Report Adverse Events to CDPH (HSC ...,AE17 - Stage 3 or 4 ulcer acquired after admis...,No,FTR AE,NaN,$700,$700
3,JOHN MUIR MEDICAL CENTER-CONCORD CAMPUS,GENERAL ACUTE CARE HOSPITAL,East Bay,20Feb2008,20004706,Closed,Failure to Report Adverse Events to CDPH (HSC ...,AE17 - Stage 3 or 4 ulcer acquired after admis...,Yes,FTR AE,NaN,"$2,800","$2,800"
4,JOHN MUIR MEDICAL CENTER-WALNUT CREEK CAMPUS,GENERAL ACUTE CARE HOSPITAL,East Bay,25Feb2008,20004711,Closed,Failure to Report Adverse Events to CDPH (HSC ...,AE17 - Stage 3 or 4 ulcer acquired after admis...,Yes,FTR AE,NaN,$700,$700


In [2]:
# Determine whether any columns need type conversions
for series in range(13): print(type(enforce.iloc[:, series].values[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'str'>


In [3]:
# Convert dates and currencies to numbers
enforce['PENALTY_ISSUE_DATE'] = pd.to_datetime(enforce['PENALTY_ISSUE_DATE'])
enforce['TOTAL_AMOUNT_DUE_FINAL'] = enforce['TOTAL_AMOUNT_DUE_FINAL'].replace('[\$,]', '', regex=True).astype(float)
enforce['TOTAL_AMOUNT_INITIAL'] = enforce['TOTAL_AMOUNT_INITIAL'].replace('[\$,]', '', regex=True).astype(float)

In [4]:
# Check for null values -- the only null values are in 'class-assessed-final', which is fine because the class doesn't
# always change before and after an appeal.

enforce.isnull().sum()

FACILITY_NAME                 0
FAC_FDR                       0
DISTRICT_OFFICE               0
PENALTY_ISSUE_DATE            0
PENALTY_NUMBER                0
DISPOSITION                   0
PENALTY_DETAIL                0
PENALTY_CATEGORY              0
APPEALED                      0
CLASS_ASSESSED_INITIAL        0
CLASS_ASSESSED_FINAL      12202
TOTAL_AMOUNT_INITIAL          0
TOTAL_AMOUNT_DUE_FINAL        0
dtype: int64

In [5]:
# Save cleaned database for later notebooks
enforce.to_csv("../data/interim/enforce.csv")

In [6]:
# Load secondary database with detailed narratives
narrative = pd.read_csv("../data/raw/citation_narr.csv", encoding='unicode-escape')
narrative.head()

,FACID,FACILITY_NAME,PENALTY_NUMBER,CLASS_ASSESSED_INITIAL,PENALTY_ISSUE_DATE,EVENTID,NARRATIVE LENGTH,NARRATIVE
0,20000132,Fremont HealthCare Center,20008964,B,01-Feb-12,V6XV11,4029,F323 483.25(h) FREE OF ACCIDENT HAZARDS/SUPERV...
1,20000054,Willow Tree Nursing Center,20009068,B,02-Mar-12,K04O11,2678,Title 22 72520 (a) If a patient of a skilled n...
2,140000051,Kindred Nursing and Rehabilitation - Ygnacio V...,20009069,B,02-Mar-12,FGDY11,7868,483.12(b) (3) Permitting Resident to Return to...
3,20000026,"Bay View Rehabilitation Hospital, LLC",20009078,AA,05-Mar-12,8JFW11,9084,483.25 PROVIDE CARE/SERVICES FOR HIGHEST WELL ...
4,140000105,Lone Tree Convalescent Hospital,20009082,B,06-Mar-12,PUKB11,5560,T22 DIV5 CH3 ART3-72311(a)(1)(A) Nursing Servi...


In [7]:
# Convert facility names to uppercase to assist with merge of main database and detailed narrative database
narrative['FACILITY_NAME'] = narrative['FACILITY_NAME'].str.upper()
narrative_basics = narrative[['PENALTY_NUMBER', 'FACILITY_NAME', 'NARRATIVE']]
narrative_basics.head()

,PENALTY_NUMBER,FACILITY_NAME,NARRATIVE
0,20008964,FREMONT HEALTHCARE CENTER,F323 483.25(h) FREE OF ACCIDENT HAZARDS/SUPERV...
1,20009068,WILLOW TREE NURSING CENTER,Title 22 72520 (a) If a patient of a skilled n...
2,20009069,KINDRED NURSING AND REHABILITATION - YGNACIO V...,483.12(b) (3) Permitting Resident to Return to...
3,20009078,"BAY VIEW REHABILITATION HOSPITAL, LLC",483.25 PROVIDE CARE/SERVICES FOR HIGHEST WELL ...
4,20009082,LONE TREE CONVALESCENT HOSPITAL,T22 DIV5 CH3 ART3-72311(a)(1)(A) Nursing Servi...


In [8]:
# Merge main database and narrative database

working = narrative_basics.merge(enforce, how="left", on=["PENALTY_NUMBER"])
working = working[['PENALTY_NUMBER', 'FACILITY_NAME_x', 'PENALTY_ISSUE_DATE', 'TOTAL_AMOUNT_DUE_FINAL', 'NARRATIVE']]
working.columns = ['NUM', 'FACILITY', 'DATE', 'FINE', 'NARRATIVE']
working.head()

,NUM,FACILITY,DATE,FINE,NARRATIVE
0,20008964,FREMONT HEALTHCARE CENTER,2012-02-01,750.0,F323 483.25(h) FREE OF ACCIDENT HAZARDS/SUPERV...
1,20009068,WILLOW TREE NURSING CENTER,2012-03-02,750.0,Title 22 72520 (a) If a patient of a skilled n...
2,20009069,KINDRED NURSING AND REHABILITATION - YGNACIO V...,2012-03-02,750.0,483.12(b) (3) Permitting Resident to Return to...
3,20009078,"BAY VIEW REHABILITATION HOSPITAL, LLC",2012-03-05,37500.0,483.25 PROVIDE CARE/SERVICES FOR HIGHEST WELL ...
4,20009082,LONE TREE CONVALESCENT HOSPITAL,2012-03-06,600.0,T22 DIV5 CH3 ART3-72311(a)(1)(A) Nursing Servi...


In [9]:
# Check for null values in merged database
working.isnull().sum()

NUM          0
FACILITY     0
DATE         2
FINE         2
NARRATIVE    0
dtype: int64

In [10]:
#See which rows are null in merged database
working[working['FINE'].isnull()]

,NUM,FACILITY,DATE,FINE,NARRATIVE
708,70012573,LOS GATOS MEADOWS GERIATRIC HOSPITAL,NaT,NaN,"F226, 483.13(c) DEVELOP/IMPLEMENT ABUSE/NEGLEC..."
2355,940012943,BEL TOOREN VILLA CONVALESCENT HOSPITAL,NaT,NaN,?483.25 Quality of Care\nEach resident must re...


In [11]:
# Drop the 2 out of 2,800 rows that are null rather than investigate why the fine is missing
working = working.drop([708, 2355])

In [12]:
# Save the merged database for more work in future notebooks
working.to_csv("../data/interim/working.csv")